In [1]:
import numpy as np
import scipy as sp
import pandas as pd

import matplotlib as mtlb
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()


### Reading Dataset

In [28]:
X_train = pd.read_csv("X_train_Wwou3IE.csv")
y_train = pd.read_csv("y_train_jJtXgMX.csv")
df = pd.concat([X_train, y_train["spot_id_delta"]], axis=1).rename(columns={"nucelear_power_available":"nuclear_power_available"})


#### Dataset overview

In [29]:
df

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nuclear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,spot_id_delta
0,2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,NaN,-36.874770
1,2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,NaN,-12.643588
2,2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,NaN,-1.950193
3,2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,NaN,1.938272
4,2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,NaN,0.199907
...,...,...,...,...,...,...,...,...,...,...,...
10600,2023-03-29 19:00:00+02:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.11,6.029303
10601,2023-03-29 20:00:00+02:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.66,13.576177
10602,2023-03-29 21:00:00+02:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.01,17.478945
10603,2023-03-29 22:00:00+02:00,47967.0,3386.0,11952.0,38320.0,9636.0,0.0,156.669189,0.000000,136.74,17.559407


In [30]:
df.shape

(10605, 11)

**Insight**:

The Dataframe has 10605 rows and 11 columns (with one which is the interest variable)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10605 entries, 0 to 10604
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   DELIVERY_START                 10605 non-null  object 
 1   load_forecast                  9318 non-null   float64
 2   coal_power_available           10604 non-null  float64
 3   gas_power_available            10604 non-null  float64
 4   nuclear_power_available        10604 non-null  float64
 5   wind_power_forecasts_average   10581 non-null  float64
 6   solar_power_forecasts_average  10581 non-null  float64
 7   wind_power_forecasts_std       10581 non-null  float64
 8   solar_power_forecasts_std      10581 non-null  float64
 9   predicted_spot_price           1846 non-null   float64
 10  spot_id_delta                  10605 non-null  float64
dtypes: float64(10), object(1)
memory usage: 911.5+ KB


**Insight**:

- Most of the variables are Float 
- Delivery start variable is String type and will not be useful for the regression/classification task

In [22]:
# Displays some raws of the dataframe
print("\nSample of the dataset:")
df.sample(10)


Sample of the dataset:


,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,spot_id_delta
644,2022-01-27 22:00:00+01:00,72411.0,3386.0,11487.0,49697.0,3122.0,0.0,249.495979,0.000000,NaN,1.690682
9929,2023-03-01 19:00:00+01:00,73626.0,3386.0,11950.0,39615.0,6246.0,0.0,175.634584,0.000000,184.39,-3.179184
1364,2022-02-28 13:00:00+01:00,65750.0,3386.0,11952.0,43290.0,1772.0,7573.0,15.159824,29.399491,NaN,9.761202
4182,2022-07-01 08:00:00+02:00,NaN,2226.0,10555.0,28620.0,1500.0,2769.0,52.120054,11.251144,NaN,-19.101421
8315,2022-12-23 13:00:00+01:00,52635.0,2226.0,11945.0,41115.0,NaN,2959.0,NaN,22.345256,NaN,-17.054225
6523,2022-10-09 21:00:00+02:00,41463.0,2226.0,11277.0,30140.0,3725.0,0.0,298.600677,0.000000,NaN,-43.775874
4588,2022-07-18 06:00:00+02:00,NaN,2226.0,11140.0,25640.0,3542.0,252.0,97.414903,1.294786,NaN,44.155972
5891,2022-09-10 13:00:00+02:00,43304.0,2226.0,10819.0,24235.0,3993.0,7688.0,90.995026,45.476849,NaN,-28.590965
3784,2022-06-13 18:00:00+02:00,48683.0,2226.0,10255.0,30150.0,1468.0,4916.0,38.311785,26.746547,NaN,-1.487979
3862,2022-06-18 00:00:00+02:00,44224.0,2226.0,10085.0,28165.0,1854.0,0.0,40.793318,0.000000,NaN,-10.447535


**Insight**:

Some Nan values must be further processed


In [33]:
#Display the dataset of raws of df with at least one Nan value
df[df.isnull().any(axis=1)]

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nuclear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,spot_id_delta
0,2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.0,NaN,-36.874770
1,2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.0,NaN,-12.643588
2,2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.0,NaN,-1.950193
3,2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.0,NaN,1.938272
4,2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.0,NaN,0.199907
...,...,...,...,...,...,...,...,...,...,...,...
10554,2023-03-27 21:00:00+02:00,52863.0,3386.0,11952.0,36990.0,1837.0,0.0,136.999062,0.0,NaN,1.540616
10555,2023-03-27 22:00:00+02:00,52783.0,3386.0,11952.0,36990.0,1812.0,0.0,128.421281,0.0,NaN,-1.485172
10556,2023-03-27 23:00:00+02:00,53274.0,3386.0,11952.0,36990.0,1854.0,0.0,106.112440,0.0,NaN,-6.687949
10557,2023-03-28 00:00:00+02:00,50798.0,3386.0,11952.0,36990.0,2050.0,0.0,76.348188,0.0,NaN,7.277778


**Insight**:

- The dataset is of very low quality with near to 80% of its rows filled with Nan values.
- It could be interesting to see how much are there for each column

In [37]:
def count_nan_by_column(dataframe):
    "Displays the number of Nan values for each column"

    for column in df.columns:
        col_value = dataframe[column]
        number_of_nan = col_value.isnull().sum()
        print(f"Number of Nan values for '{column}' column: {number_of_nan}")

    return 0

print("Here the number of Nan values for each column:")
print("______________________________________________")
count_nan_by_column(df)
    

Here the number of Nan values for each column:
______________________________________________
Number of Nan values for 'DELIVERY_START' column: 0
Number of Nan values for 'load_forecast' column: 1287
Number of Nan values for 'coal_power_available' column: 1
Number of Nan values for 'gas_power_available' column: 1
Number of Nan values for 'nuclear_power_available' column: 1
Number of Nan values for 'wind_power_forecasts_average' column: 24
Number of Nan values for 'solar_power_forecasts_average' column: 24
Number of Nan values for 'wind_power_forecasts_std' column: 24
Number of Nan values for 'solar_power_forecasts_std' column: 24
Number of Nan values for 'predicted_spot_price' column: 8759
Number of Nan values for 'spot_id_delta' column: 0


0

**Insight**:

- Most of the Nan values are in the "load_forecast" column with 1287 Nan values and in "predicted_spot_price" column with 8759 Nan values.
- Since there are only 8783 rows with Nan values, there at leasrt roughly 1000 rough with 2 Nan values
- These two columns must be analysed and processed properly

In [38]:
# Summarized Statistical description of the dataset
print("\nStatistical description:")
df.describe()




Statistical description:


,load_forecast,coal_power_available,gas_power_available,nuclear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,spot_id_delta
count,9318.000000,10604.000000,10604.000000,10604.000000,10581.000000,10581.000000,10581.000000,10581.000000,1846.000000,10605.000000
mean,53725.132968,2819.781592,11463.220671,35393.282252,4512.989415,1919.281353,125.399422,24.997384,140.256024,0.943358
std,11274.194929,495.103588,483.799154,7619.820162,3349.419356,2706.963383,132.152875,43.083977,33.922669,38.093819
min,30515.000000,1646.000000,9769.000000,22285.000000,592.000000,0.000000,1.830211,0.000000,42.420000,-1567.353547
25%,45336.500000,2226.000000,11140.000000,29005.000000,2036.000000,0.000000,46.818974,0.000000,119.405000,-11.934152
50%,51846.000000,2806.000000,11487.000000,32470.000000,3319.000000,149.000000,86.700163,3.507229,140.040000,0.059125
75%,62091.750000,3386.000000,11945.000000,43055.000000,5951.000000,3507.000000,155.325644,35.137019,160.765000,12.190489
max,85145.000000,3391.000000,11963.000000,50122.000000,15167.000000,10127.000000,1871.571992,745.261316,254.190000,658.961329


**Insight**
